In [6]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langchain.callbacks.tracers import LangChainTracer
from langgraph.checkpoint.sqlite import SqliteSaver

langsmith = LangChainTracer()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

_ = load_dotenv()


In [7]:
@tool
def even_handler(n: int) -> str:
    """Handle even numbers by dividing them by 2."""
    if n % 2 == 0:
        return str(n // 2)
    return "DO-NOT-KNOW"
@tool
def odd_handler(n: int) -> str:
    """Handle odd numbers by multiplying by 3 and adding 1."""
    if n % 2 != 0:
        return str(3 * n + 1)
    return "DO-NOT-KNOW"

tools = [even_handler, odd_handler]

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [9]:

class Agent:

    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("even", self.even_handler)
        graph.add_node("odd", self.odd_handler)
        graph.add_edge("even","llm")
        graph.add_edge("odd", "llm")
        graph.set_entry_point("llm")

        graph.add_conditional_edges(
            "llm",
            self.decide_action,
            {
                "even": "even",
                "odd": "odd",
                "end": END
            }
        )
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def decide_action(self, state: AgentState):
        print("\n--- In decide_action ---")
        last_message = state['messages'][-1]
        print(f"Last message: {last_message}")
        if not last_message.tool_calls:
            print("No tool calls, ending.")
            return "end"
        
        tool_call = last_message.tool_calls[0]
        print(f"Tool call: {tool_call}")
        tool_name = tool_call.get('name')
        print(f"Tool name: {tool_name}")
        
        if tool_name == "even_handler":
            print("Decided: even")
            return "even"
        elif tool_name == "odd_handler":
            print("Decided: odd")
            return "odd"
        else:
            print("Decided: end")
            return "end"

    def call_llm(self, state: AgentState):
        print("\n--- In call_llm ---")
        messages = state['messages']
        print(f"Input messages: {messages}")
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        print(f"LLM output: {message}")
        return {'messages': [message]}

    def even_handler(self, state: AgentState):
        print("\n--- In even_handler ---")
        last_message = state['messages'][-1]
        print(f"Last message: {last_message}")
        tool_call = last_message.tool_calls[0]
        print(f"Tool call: {tool_call}")
        args = tool_call.get('args', {})
        print(f"Args: {args}")
        
        n = int(args.get('n', 0))
        print(f"Number to process: {n}")
        
        if n % 2 == 0:
            result = str(n // 2)
        else:
            result = "DO-NOT-KNOW"
        print(f"Result: {result}")
        return {'messages': [ToolMessage(content=result, name="even_handler", tool_call_id=tool_call.get('id', ''))]}
    
    def odd_handler(self, state: AgentState):
        print("\n--- In odd_handler ---")
        last_message = state['messages'][-1]
        print(f"Last message: {last_message}")
        tool_call = last_message.tool_calls[0]
        print(f"Tool call: {tool_call}")
        args = tool_call.get('args', {})
        print(f"Args: {args}")
        
        n = int(args.get('n', 0))
        print(f"Number to process: {n}")
        
        if n % 2 != 0:
            result = str(3 * n + 1)
        else:
            result = "DO-NOT-KNOW"
        print(f"Result: {result}")
        return {'messages': [ToolMessage(content=result, name="odd_handler", tool_call_id=tool_call.get('id', ''))]}



In [16]:
import os
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
system_message="""
        You will receive a list of numbers from the user.
        Your goal is to apply a transformation to each number.
        However you do not know how to do this transformation. 
        You can simply send each number FROM THE GIVEN LIST
        and the tools will do the transforamtion for you
        with the appropriate transformation applied.
        IMPORTANT: only send one number at a time, concisely, say nothing else.
        Once you have accomplished your goal, say DONE and show the result.
        Start by asking the user for the list of numbers.
        Your tools will be :
        even_handler use this for even numbers
        odd_handler use this for odd numbers


        """

model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key)


In [18]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, tools, checkpointer=memory, system=system_message)
    messages = [HumanMessage(content="1 3 6 9")]
    messages2 = [HumanMessage(content="I want to add 4 5 to the original sequence and then see what is the full transformed list")]
    
    thread = {"configurable": {"thread_id": "1"}}
    thread2 = {"configurable": {"thread_id": "2"}}
    
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

    for event in abot.graph.stream({"messages": messages2}, thread):
        for v in event.values():
            print(v['messages'])
            
    for event in abot.graph.stream({"messages": messages2}, thread2):
        for v in event.values():
            print(v['messages'])


--- In call_llm ---
Input messages: [HumanMessage(content='1 3 6 9', additional_kwargs={}, response_metadata={})]
LLM output: content=[{'text': "Thank you for providing the list of numbers: 1, 3, 6, and 9. I'll now process each number using the appropriate handler function.", 'type': 'text'}, {'id': 'toolu_01DNZ33amuNJ4aT1hy6S4rJe', 'input': {'n': 1}, 'name': 'odd_handler', 'type': 'tool_use'}] additional_kwargs={} response_metadata={'id': 'msg_014KsYaXUfogG3bvxoy2eZBH', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 581, 'output_tokens': 88}} id='run-9382b590-e325-4b6a-a1ed-4fb135b6b0ad-0' tool_calls=[{'name': 'odd_handler', 'args': {'n': 1}, 'id': 'toolu_01DNZ33amuNJ4aT1hy6S4rJe', 'type': 'tool_call'}] usage_metadata={'input_tokens': 581, 'output_tokens': 88, 'total_tokens': 669}

--- In decide_action ---
Last message: content=[{'text': "Thank you for providing the list of numbers: 1, 3, 6, and 9. I'll now process e

## Streaming tokens

In [27]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

with AsyncSqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, tools, checkpointer=memory, system=system_message)
    messages = [HumanMessage(content="1 3 6 9")]
    thread = {"configurable": {"thread_id": "4"}}
    async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")

TypeError: '_AsyncGeneratorContextManager' object does not support the context manager protocol

In [25]:
async def stream_tokens():
    print("="*50)
    print("Streaming tokens")
    print("="*50)

    memory = AsyncSqliteSaver.from_conn_string(":memory:")
    abot = Agent(model, tools, system=system_message, checkpointer=memory)

    messages = [HumanMessage(content="1 3 5")]
    thread = {"configurable": {"thread_id": "4"}}
    async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="|")



In [26]:
import asyncio

asyncio.run(stream_tokens())

c:\Egyetem\7Felev\Szakdolgozat\number_game\.venv\Lib\site-packages\pygments\regexopt.py:77: RuntimeWarning: coroutine 'stream_tokens' was never awaited
  '|'.join(regex_opt_inner(list(group[1]), '')


RuntimeError: asyncio.run() cannot be called from a running event loop